In [1]:
from tinyml import *


In [2]:
Model = CreateModel()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                23072     
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 2)                 34        
                                                                 
Total params: 23,634
Trainable params: 23,634
Non-trainable params: 0
_________________________________________________________________


In [3]:
x, y = ReadTrainingData()
TrainModel(Model, x, y)

Epoch 1/200
6/6 [==============================] - 1s 61ms/step - loss: 0.0338 - categorical_accuracy: 1.0000 - val_loss: 3.3762e-04 - val_categorical_accuracy: 1.0000
Epoch 2/200
6/6 [==============================] - 0s 9ms/step - loss: 8.1729e-05 - categorical_accuracy: 1.0000 - val_loss: 6.4333e-05 - val_categorical_accuracy: 1.0000
Epoch 3/200
6/6 [==============================] - 0s 8ms/step - loss: 1.1689e-05 - categorical_accuracy: 1.0000 - val_loss: 2.4793e-05 - val_categorical_accuracy: 1.0000
Epoch 4/200
6/6 [==============================] - 0s 7ms/step - loss: 5.5902e-06 - categorical_accuracy: 1.0000 - val_loss: 1.4142e-05 - val_categorical_accuracy: 1.0000
Epoch 5/200
6/6 [==============================] - 0s 10ms/step - loss: 3.3807e-06 - categorical_accuracy: 1.0000 - val_loss: 1.0154e-05 - val_categorical_accuracy: 1.0000
Epoch 6/200
6/6 [==============================] - 0s 8ms/step - loss: 2.7307e-06 - categorical_accuracy: 1.0000 - val_loss: 8.3227e-06 - val_categ

Epoch 49/200
6/6 [==============================] - 0s 8ms/step - loss: 9.3703e-07 - categorical_accuracy: 1.0000 - val_loss: 3.3107e-06 - val_categorical_accuracy: 1.0000
Epoch 50/200
6/6 [==============================] - 0s 7ms/step - loss: 9.2179e-07 - categorical_accuracy: 1.0000 - val_loss: 3.2782e-06 - val_categorical_accuracy: 1.0000
Epoch 51/200
6/6 [==============================] - 0s 10ms/step - loss: 9.1070e-07 - categorical_accuracy: 1.0000 - val_loss: 3.2511e-06 - val_categorical_accuracy: 1.0000
Epoch 52/200
6/6 [==============================] - 0s 9ms/step - loss: 9.0099e-07 - categorical_accuracy: 1.0000 - val_loss: 3.2023e-06 - val_categorical_accuracy: 1.0000
Epoch 53/200
6/6 [==============================] - 0s 9ms/step - loss: 8.9129e-07 - categorical_accuracy: 1.0000 - val_loss: 3.1644e-06 - val_categorical_accuracy: 1.0000
Epoch 54/200
6/6 [==============================] - 0s 10ms/step - loss: 8.7882e-07 - categorical_accuracy: 1.0000 - val_loss: 3.1048e-06 -

6/6 [==============================] - 0s 7ms/step - loss: 5.8911e-07 - categorical_accuracy: 1.0000 - val_loss: 1.8857e-06 - val_categorical_accuracy: 1.0000
Epoch 97/200
6/6 [==============================] - 0s 8ms/step - loss: 5.8357e-07 - categorical_accuracy: 1.0000 - val_loss: 1.8640e-06 - val_categorical_accuracy: 1.0000
Epoch 98/200
6/6 [==============================] - 0s 6ms/step - loss: 5.7664e-07 - categorical_accuracy: 1.0000 - val_loss: 1.8423e-06 - val_categorical_accuracy: 1.0000
Epoch 99/200
6/6 [==============================] - 0s 8ms/step - loss: 5.7248e-07 - categorical_accuracy: 1.0000 - val_loss: 1.8206e-06 - val_categorical_accuracy: 1.0000
Epoch 100/200
6/6 [==============================] - 0s 6ms/step - loss: 5.6694e-07 - categorical_accuracy: 1.0000 - val_loss: 1.8044e-06 - val_categorical_accuracy: 1.0000
Epoch 101/200
6/6 [==============================] - 0s 7ms/step - loss: 5.6416e-07 - categorical_accuracy: 1.0000 - val_loss: 1.7827e-06 - val_categori

6/6 [==============================] - 0s 8ms/step - loss: 4.2139e-07 - categorical_accuracy: 1.0000 - val_loss: 1.1542e-06 - val_categorical_accuracy: 1.0000
Epoch 144/200
6/6 [==============================] - 0s 7ms/step - loss: 4.1862e-07 - categorical_accuracy: 1.0000 - val_loss: 1.1487e-06 - val_categorical_accuracy: 1.0000
Epoch 145/200
6/6 [==============================] - 0s 8ms/step - loss: 4.1723e-07 - categorical_accuracy: 1.0000 - val_loss: 1.1433e-06 - val_categorical_accuracy: 1.0000
Epoch 146/200
6/6 [==============================] - 0s 8ms/step - loss: 4.1446e-07 - categorical_accuracy: 1.0000 - val_loss: 1.1271e-06 - val_categorical_accuracy: 1.0000
Epoch 147/200
6/6 [==============================] - 0s 9ms/step - loss: 4.1169e-07 - categorical_accuracy: 1.0000 - val_loss: 1.1216e-06 - val_categorical_accuracy: 1.0000
Epoch 148/200
6/6 [==============================] - 0s 7ms/step - loss: 4.0891e-07 - categorical_accuracy: 1.0000 - val_loss: 1.1108e-06 - val_categ

6/6 [==============================] - 0s 8ms/step - loss: 3.2713e-07 - categorical_accuracy: 1.0000 - val_loss: 8.5072e-07 - val_categorical_accuracy: 1.0000
Epoch 191/200
6/6 [==============================] - 0s 7ms/step - loss: 3.2713e-07 - categorical_accuracy: 1.0000 - val_loss: 8.4530e-07 - val_categorical_accuracy: 1.0000
Epoch 192/200
6/6 [==============================] - 0s 7ms/step - loss: 3.2713e-07 - categorical_accuracy: 1.0000 - val_loss: 8.3988e-07 - val_categorical_accuracy: 1.0000
Epoch 193/200
6/6 [==============================] - 0s 8ms/step - loss: 3.2575e-07 - categorical_accuracy: 1.0000 - val_loss: 8.3988e-07 - val_categorical_accuracy: 1.0000
Epoch 194/200
6/6 [==============================] - 0s 7ms/step - loss: 3.2159e-07 - categorical_accuracy: 1.0000 - val_loss: 8.3446e-07 - val_categorical_accuracy: 1.0000
Epoch 195/200
6/6 [==============================] - 0s 7ms/step - loss: 3.2159e-07 - categorical_accuracy: 1.0000 - val_loss: 8.2904e-07 - val_categ

In [ ]:
ConvertModel(Model)

In [ ]:
Hex2H(Model, "model")